In [1]:
import tensorflow as tf
print("TF version: ", tf.__version__)

TF version:  2.3.1


In [2]:
from transformers import AutoConfig, AutoModel, TFAutoModel, AutoTokenizer

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from data_preprocessing import remove_stop_words, text_normalization, add_data_portion

from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import regularizers

#### Load Dataset

In [5]:
spanish_dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
print(spanish_dataset.shape)

(2571, 2)


In [6]:
max_length_sequence = 500

#### Load BETO Model

In [10]:
config = AutoConfig.from_pretrained('../data/bert_beto/config.json')
#model = AutoModel.from_pretrained('../data/bert_beto/model.ckpt-2000000.index', from_tf=True, config=config)
beto = TFAutoModel.from_pretrained('../data/bert_beto/pytorch_model.bin', from_pt=True, config=config)
print(config, beto)

{
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 31002
}


In [11]:
#tokenizer = AutoTokenizer.from_pretrained('../data/bert_beto/vocab.txt')
tokenizer = AutoTokenizer.from_pretrained('../data/bert_beto/vocab.txt', do_lower_case=True, add_special_tokens=True,
                                          max_length=max_length_sequence, pad_to_max_length=True)
tokenizer

#### Prepare Data

In [12]:
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.encode(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [13]:
def normalize_and_tokenize_data(df, max_length_sequence):
    text_normalization(df) # Normalize text
    remove_stop_words(df, language = 'spanish', get_tokenize = False) # Remove stop words [and Tokenize texts]
    
    all_sentences = df['text'].values
    all_words = []
    for sent in all_sentences:
        temp = []
        temp.append('[CLS]')
        i = 0
        for w in tokenizer.tokenize(sent):
            i+=1
            if i == (max_length_sequence - 1): break
            temp.append(w)
        temp.append('[SEP]')
        all_words.append(temp)

    return all_words

In [14]:
all_words = normalize_and_tokenize_data(spanish_dataset, max_length_sequence)

input_ids = np.zeros((len(all_words), max_length_sequence))
input_masks = np.zeros((len(all_words), max_length_sequence))
input_segments = np.zeros((len(all_words), max_length_sequence))

for i in range(len(all_words)):
    input_ids[i,:] = np.array(get_ids(all_words[i], tokenizer, max_length_sequence)).reshape(1,-1)
    input_masks[i,:] = np.array(get_masks(all_words[i], max_length_sequence)).reshape(1,-1)
    input_segments[i,:] = np.array(get_segments(all_words[i], max_length_sequence)).reshape(1,-1)

input_ids = input_ids.astype(int)

In [15]:
print(input_ids.shape, input_masks.shape, input_segments.shape)

(2571, 500) (2571, 500) (2571, 500)


In [ ]:
print(input_ids[0])
print(input_masks[0])
print(input_segments[0])

#### Models

In [17]:
batch_size = 32

In [18]:
def create_model_CNN(max_length_sequence, filters, kernel_size, dense_units, l2_kernel):
    
    input_word_ids = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="segment_ids")
    
    embedding_layer = beto(input_word_ids, attention_mask=input_mask, token_type_ids=segment_ids)
    
    X = Conv1D(filters = filters, kernel_size = kernel_size, activation = 'relu',
              kernel_regularizer = regularizers.l2(l2_kernel))(embedding_layer[0])
    X = MaxPooling1D(pool_size = 2)(X)
    X = Flatten()(X)
    X = Dense(units = dense_units, activation = 'relu')(X)
    X = Dense(units = 1, activation = 'sigmoid')(X)
                          
    model = Model(inputs = [input_word_ids, input_mask, segment_ids], outputs = X)
    
    for layer in model.layers[:4]:
        layer.trainable = False
    
    return(model, embedding_layer)

In [26]:
def create_model_RNN(max_length_sequence, lstm_units, l2_kernel, l2_recurrent, l2_activity, dropout):
    
    input_word_ids = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_length_sequence, ), dtype=tf.int32, name="segment_ids")
    
    embedding_layer = beto(input_word_ids, attention_mask=input_mask, token_type_ids=segment_ids)
    
    X = LSTM(units = lstm_units, return_sequences = False,
            kernel_regularizer = regularizers.l2(l2_kernel),
            recurrent_regularizer = regularizers.l2(l2_recurrent),
            activity_regularizer = regularizers.l2(l2_activity))(embedding_layer[0])
    
    X = Dropout(rate = dropout)(X)
    X = Dense(units = 1, activation = 'sigmoid')(X)
                          
    model = Model(inputs = [input_word_ids, input_mask, segment_ids], outputs = X)
    
    for layer in model.layers[:4]:
        layer.trainable = False
                          
    return(model)

#### Execute Models

In [20]:
epochs = 10

In [21]:
test_size = 0.2
Y = spanish_dataset.label.values

input_ids_tr, input_ids_te, input_masks_tr, input_masks_te, input_segments_tr, input_segments_te, y_tr, y_te = train_test_split(
    input_ids, input_masks, input_segments, Y, test_size = test_size, shuffle = True)

In [22]:
print(len(input_ids_tr), len(input_ids_te))
print(len(input_masks_tr), len(input_masks_te))
print(len(input_segments_tr), len(input_segments_te))
print(len(y_tr), len(y_te))

2056 515
2056 515
2056 515
2056 515


CNN

In [23]:
CNN, emb_layer = create_model_CNN(max_length_sequence = max_length_sequence, filters = 16, kernel_size = 10, dense_units = 4, l2_kernel = 0)
CNN.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
CNN.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 500, 768), ( 109850880   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
CNN.fit([input_ids_tr, input_masks_tr, input_segments_tr], y_tr, epochs = epochs, batch_size = batch_size, shuffle = True) # Fit model
loss, acc = CNN.evaluate([input_ids_te, input_masks_te, input_segments_te], y_te) # Evaluate model
print(loss, round(acc, 3))
CNN.save_weights('../data/Weights/BETO_CNN.h5')

RNN

In [28]:
RNN = create_model_RNN(max_length_sequence = max_length_sequence, lstm_units = 8, l2_kernel = 0, l2_recurrent = 0, l2_activity = 0, dropout = 0.5)
RNN.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
RNN.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   ((None, 500, 768), ( 109850880   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
RNN.fit([input_ids_tr, input_masks_tr, input_segments_tr], y_tr, epochs = epochs, batch_size = batch_size, shuffle = True) # Fit model
loss, acc = RNN.evaluate([input_ids_te, input_masks_te, input_segments_te], y_te) # Evaluate model
print(loss, round(acc, 3))
RNN.save_weights('../data/Weights/BETO_RNN.h5')